In [1]:
# author: René Kopeinig
# script: Tasseled Cap Transformation for atmospherical adjusted Landsat 8 imagery
# description: Tasseled Cap Transformation for Landsat 8 TOA imagery
#              based on the scientfic work "Derivation of a tasselled cap transformation based 
#              on Landsat 8 at-satellite reflectance" by M.Baigab, L.Zhang, T.Shuai & Q.Tong (2014).
#              The bands of the output image are the brightness index, greenness index and wetness index.

In [1]:
from IPython.display import Image
import ee

ee.Initialize()

In [2]:
# Input Landsat 8 TOA image in Oaxaca, Veracruz, Tabasco and Chiapas México
image = ee.Image('LANDSAT/LC8_L1T_TOA/LC80230482016032LGN00')

In [3]:
# Function
def tasseled_cap_transformation(image):
    b = image.select("B2", "B3", "B4", "B5", "B6", "B7");
    #Coefficients are only for Landsat 8 TOA imagery
    brightness_coefficents= ee.Image([0.3029, 0.2786, 0.4733, 0.5599, 0.508, 0.1872])
    greenness_coefficents= ee.Image([-0.2941, -0.243, -0.5424, 0.7276, 0.0713, -0.1608]);
    wetness_coefficents= ee.Image([0.1511, 0.1973, 0.3283, 0.3407, -0.7117, -0.4559]);
    fourth_coefficents= ee.Image([-0.8239, 0.0849, 0.4396, -0.058, 0.2013, -0.2773]);
    fifth_coefficents= ee.Image([-0.3294, 0.0557, 0.1056, 0.1855, -0.4349, 0.8085]);
    sixth_coefficents= ee.Image([0.1079, -0.9023, 0.4119, 0.0575, -0.0259, 0.0252]);
    
    #Calculate tasseled cap transformation
    brightness = image.expression(
        '(B * BRIGHTNESS)',
        {
            'B':b,
            'BRIGHTNESS': brightness_coefficents
        })
    greenness = image.expression(
        '(B * GREENNESS)',
        {
            'B':b,
            'GREENNESS': greenness_coefficents
        })
    wetness = image.expression(
        '(B * WETNESS)',
        {
            'B':b,
            'WETNESS': wetness_coefficents
        })
    fourth = image.expression(
        '(B * FOURTH)',
        {
            'B':b,
            'FOURTH': fourth_coefficents
        })
    fifth = image.expression(
        '(B * FIFTH)',
        {
            'B':b,
            'FIFTH': fifth_coefficents
        })
    sixth = image.expression(
        '(B * SIXTH)',
        {
            'B':b,
            'SIXTH': sixth_coefficents
        })
    bright = brightness.reduce(ee.call("Reducer.sum"));
    green = greenness.reduce(ee.call("Reducer.sum"));
    wet = wetness.reduce(ee.call("Reducer.sum"));
    four = fourth.reduce(ee.call("Reducer.sum"));
    five = fifth.reduce(ee.call("Reducer.sum"));
    six = sixth.reduce(ee.call("Reducer.sum"));
    tasseled_cap = ee.Image(bright).addBands(green).addBands(wet).addBands(four).addBands(five).addBands(six)
    return tasseled_cap.rename('brightness','greenness','wetness','fourth','fifth','sixth')

In [4]:
# Execute 'Tasseled cap Transformation'
tasseled_cap = tasseled_cap_transformation(image)

In [5]:
# Display result
region = image.geometry().getInfo()['coordinates']
thumbnail = tasseled_cap.getThumbUrl({'min':-1,'max':1,'region':region})
Image(url=thumbnail)

In [6]:
tasseled_cap.bandNames().getInfo()

['brightness', 'greenness', 'wetness', 'fourth', 'fifth', 'sixth']